In [ ]:
import numpy as np

notebooks/examples/01_lie_se3.ipynb

# Q1 - Coordinate Transforms

In [ ]:
# Set as a 3x3 rotation matrix R_{CW} that rotates 30 degrees around the y-axis
rotation_CW = np.array([
    [np.cos(np.pi/6), 0, np.sin(np.pi/6)],
    [0, 1, 0],
    [-np.sin(np.pi/6), 0, np.cos(np.pi/6)]]
    )
# Define the translation vector between coordinate frames t_{CW} = [0.5, 0.0, 1.0]^T
translate_CW = np.array([0.5, 0.0, 1.0])
print("Translate from world to camera: ", translate_CW)
# Pick a world point p_W = [1.0, 0.5, 2.0]^T
point_W = np.array([1.0, 0.5, 2.0])
print("Point in world frame: ", point_W)
# What is the coordinates of p_W in the camera frame?
point_C = rotation_CW @ (point_W - translate_CW)
print("Point in camera frame: ", point_C)
# Verify this through an inverse operation
# Invert the rotation matrix
rotation_WC = rotation_CW.T
# Project onto world frame
point_W_est = (rotation_WC @ point_C) + translate_CW
print("Estimate of point back in world frame: ", point_W_est)

Translate from world to camera:  [0.5 0.  1. ]
Point in world frame:  [1.  0.5 2. ]
Point in camera frame:  [0.9330127 0.5       0.6160254]
Estimate of point back in world frame:  [1.  0.5 2. ]


# Q2 - Homogeneous transform

In [10]:
# Construct the homogeneous transform T_{CW} in SE(E) as a 4x4 matrix
translate_WC = -(rotation_CW @ translate_CW)
transform_CW = np.block([
  [rotation_CW, translate_WC.reshape(3, 1)],
  [np.zeros((1, 3)), np.ones((1,1))]]
)
print("Homogeneous transformation matrix: ", transform_CW)
# Apply it to the same point using homogeneous coordinates
point_W_h = np.hstack((point_W, 1))
point_C_h = transform_CW @ point_W_h
# Compare this with the earlier direct computation
print("Point in camera frame: ", point_C)
print("Point in camera frame (homogeneous coordinates): ", point_C_h)
# Compute T_{WC} as the inverse of T_{CW}
transform_WC = np.block([
  [rotation_WC, translate_CW.reshape(3, 1)],
  [np.zeros((1, 3)), np.ones((1,1))]
])
# Test it recovers p_W
point_W_h = transform_WC @ point_C_h
print("Point in world frame: ", point_W)
print("Point in world frame (homogeneous coordinates): ", point_W_h)

Homogeneous transformation matrix:  [[ 0.8660254  0.         0.5       -0.9330127]
 [ 0.         1.         0.        -0.       ]
 [-0.5        0.         0.8660254 -0.6160254]
 [ 0.         0.         0.         1.       ]]
Point in camera frame:  [0.9330127 0.5       0.6160254]
Point in camera frame (homogeneous coordinates):  [0.9330127 0.5       0.6160254 1.       ]
Point in world frame:  [1.  0.5 2. ]
Point in world frame (homogeneous coordinates):  [1.  0.5 2.  1. ]


# Q3 - Axis-angle and Quaternion

In [14]:
# Convert the rotation matrix into a quaternion
t = np.trace(rotation_CW)
print("Value of t (must be positive): ", np.round(t, 3))
q_w = 0.5*np.sqrt(1 + t)
q_x = (1/(4*q_w))*(rotation_CW[2][1] - rotation_CW[1][2])
q_y = (1/(4*q_w))*(rotation_CW[0][2] - rotation_CW[2][0])
q_z = (1/(4*q_w))*(rotation_CW[1][0] - rotation_CW[2][1])
q = np.hstack([q_w, q_x, q_y, q_z])
q_norm = np.linalg.norm(q)
q = q / q_norm
print("Quaternion rotation: ", np.round(q, 3))

Value of t (must be positive):  2.732
Quaternion rotation:  [0.966 0.    0.259 0.   ]
